<a href="https://colab.research.google.com/github/Fentahun60/Fentahun/blob/main/Safewater_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your CSV file
csv_file_path = '/content/drive/MyDrive/Safewater_CSV_all_new_final.csv'

# Read the CSV file into a DataFrame
try:
    df = pd.read_csv(csv_file_path)
    # Display the head of the DataFrame
    print("DataFrame head:")
    print(df.head())
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found. Please ensure the path is correct and your Drive is mounted.")
except Exception as e:
    print(f"An error occurred while reading the CSV: {e}")

In [ ]:
df.columns

In [ ]:
df = df[df['country_code_phase'] != 'AO7']
display(df.head())

In [ ]:
missing_values_percentage = df.isnull().sum() / len(df) * 100
print("Percentage of missing values per column:")
print(missing_values_percentage[missing_values_percentage > 0].sort_values(ascending=False))

In [ ]:
print(df['country_code_phase'].value_counts())

In [ ]:
selected_columns = [
    'country_code_phase',
    'interview_year',
    'Number_household_members',
    'Number_children_under5',
    'Residence',
    'total_adults',
    'source_drinking_water',
    'distance_to_water_source',
    'toilet_type',
    'Has_electricity',
    'Has_radio',
    'Has_television',
    'Has_refrigerator',
    'Numer_sleeping_rooms',
    'household_head_sex',
    'Household_head_age',
    'Household_wealth_index',
    'Household_education_level',
    'Household_head_marital_status'
]

df_sel = df[selected_columns]
display(df_sel.head())

In [ ]:
country_mapping = {
    'AO8': 'Angola', 'BF8': 'Burkina Faso', 'BJ7': 'Benin', 'BU7': 'Burundi',
    'CD8': 'Congo Democratic Republic', 'CI8': 'Côte d’Ivoire', 'CM7': 'Cameroon',
    'ET7': 'Ethiopia', 'GA7': 'Gabon', 'GH8': 'Ghana', 'GM7': 'Gambia',
    'GN7': 'Guinea', 'KE8': 'Kenya', 'LB7': 'Liberia', 'LS8': 'Lesotho',
    'MD7': 'Madagascar', 'ML8': 'Mali', 'MR7': 'Mauritania', 'MW7': 'Malawi',
    'MZ8': 'Mozambique', 'NG8': 'Nigeria', 'RW7': 'Rwanda', 'SL7': 'Sierra Leone',
    'SN8': 'Senegal', 'TD6': 'Chad', 'TZ8': 'Tanzania', 'UG7': 'Uganda',
    'ZA7': 'South Africa', 'ZM7': 'Zambia', 'ZW7': 'Zimbabwe'
}

df_sel['country_code_phase'] = df_sel['country_code_phase'].map(country_mapping)
display(df_sel.head())

In [ ]:
print(df_sel['country_code_phase'].value_counts())

In [ ]:
print(df_sel['country_code_phase'].unique())

In [ ]:
print(df_sel['source_drinking_water'].unique())

In [ ]:
improved_sources = [
    'piped into dwelling',
    'piped to yard/plot',
    'public fountain', # Assuming 'public tap/standpipe'
    "piped to neighbour's house", # Assuming 'piped to neighbor'
    'borehole with pump', # Assuming 'tube well or borehole'
    'protected well',
    'protected spring',
    'rainwater',
    'tanker truck',
    'cart with small tank',
    'bottled water',
    '21' # Numerical code for tube well or borehole
]

df_sel['source_drinking_water'] = df_sel['source_drinking_water'].apply(lambda x: 'Improved' if x in improved_sources else 'Unimproved')

# Display the new column and its value counts to verify
print(df_sel['source_drinking_water'].value_counts())
display(df_sel.head())

In [ ]:
print(df_sel['toilet_type'].unique())

In [ ]:
import numpy as np
import pandas as pd

def categorize_toilet_type(x):
    if pd.isna(x):
        return np.nan

    x_lower = str(x).lower().strip()

    # List of specific strings/codes from the unique values that correspond to 'Improved'
    improved_toilet_types = [
        'indoors: flush to piped public system',
        'indoors: flush to septic tank',
        'indoors: flush to open pit (ditch or river)', # Assuming this is equivalent to 'flush - to pit latrine'
        'inside yard: flush to septic tank',
        'inside yard: flush to piped public system',
        'inside yard: flush to open pit (ditch or river)',
        '11', # flush - to piped sewer system
        '12', # flush - to septic tank
        '13', # flush - to pit latrine
        '15', # flush - don't know where
        '21', # pit latrine - ventilated improved pit (VIP)
        '22', # pit latrine - with slab
        '41'  # composting toilet
    ]

    # Check if the current value (as lowercase string) is in our list of improved descriptions
    if x_lower in improved_toilet_types:
        return 'Improved'

    # If not found in improved descriptions, it's unimproved
    return 'Unimproved'

# Apply the function
df_sel['toilet_types'] = df_sel['toilet_type'].apply(categorize_toilet_type)

# Display the new column and its value counts to verify
print("Proportions for 'toilet_types':")
print(df_sel['toilet_types'].value_counts(normalize=True, dropna=False))
display(df_sel.head())

In [ ]:
print(df_sel['distance_to_water_source'].unique())

In [ ]:
import numpy as np
import pandas as pd

# Handle specific string values before converting to numeric
# Using .loc to prevent SettingWithCopyWarning if df_sel is a view
# If 'distance_to_water_source' is already numeric (from a previous run), these .loc replacements will not find matches.
df_sel.loc[df_sel['distance_to_water_source'] == 'on premises', 'distance_to_water_source'] = '0'
df_sel.loc[df_sel['distance_to_water_source'] == "don't know", 'distance_to_water_source'] = '31'

# Convert 'distance_to_water_source' to numeric, coercing any other errors to NaN
df_sel['distance_to_water_source'] = pd.to_numeric(df_sel['distance_to_water_source'], errors='coerce')

# Categorize 'distance_to_water_source' into two groups: '<=30' and '>30'
# Missing values will remain pd.NA in the new category column.
df_sel['distance_category'] = np.where(
    df_sel['distance_to_water_source'].isna(),
    pd.NA, # Use pd.NA instead of np.nan for compatibility with string types
    np.where(df_sel['distance_to_water_source'] <= 30, '<=30', '>30')
)

print(df_sel[['distance_to_water_source', 'distance_category']].head())
print("\nValue counts for distance_category:")
print(df_sel['distance_category'].value_counts(dropna=False))

In [ ]:
df_sel.head()

In [ ]:
print(df_sel['distance_category'].value_counts(normalize=True, dropna=False))

In [ ]:
df_sel.columns

In [ ]:
import numpy as np

df_sel['household_members_category'] = np.where(
    df_sel['Number_household_members'] <= 6,
    '1-6',
    '>6'
)

print(df_sel[['Number_household_members', 'household_members_category']].head())
print("\nValue counts for household_members_category:")
print(df_sel['household_members_category'].value_counts(dropna=False))

In [ ]:
df_sel.columns

In [ ]:
print(df_sel['Numer_sleeping_rooms'].value_counts(dropna=False))

In [ ]:
import numpy as np
import pandas as pd # Import pandas to use pd.NA

conditions = [
    df_sel['Numer_sleeping_rooms'] == 1,
    df_sel['Numer_sleeping_rooms'] == 2,
    df_sel['Numer_sleeping_rooms'] >= 3
]

choices = ['1', '2', '>=3']

# Use pd.NA for the default value to ensure type compatibility with string choices
df_sel['sleeping_rooms_category'] = np.select(conditions, choices, default=pd.NA)

print(df_sel[['Numer_sleeping_rooms', 'sleeping_rooms_category']].head())
print("\nValue counts for sleeping_rooms_category:")
print(df_sel['sleeping_rooms_category'].value_counts(dropna=False))

In [ ]:
print(df_sel['Household_head_age'].value_counts(dropna=False))

In [ ]:
import pandas as pd

# Convert 'Household_head_age' to numeric, coercing errors to NaN
df_sel['Household_head_age'] = pd.to_numeric(df_sel['Household_head_age'], errors='coerce')

# Define the bins and labels for the age categories
bins = [0, 19, 35, 45, df_sel['Household_head_age'].max() + 1]
labels = ['<20', '20-35', '36-45', '>46']

# Recategorize 'Household_head_age' into the new categories
df_sel['Household_head_age_category'] = pd.cut(
    df_sel['Household_head_age'],
    bins=bins,
    labels=labels,
    right=True,  # Bins are (min, max]
    include_lowest=True # Include the lowest value (0)
)

# Display the head of the relevant columns and value counts
print(df_sel[['Household_head_age', 'Household_head_age_category']].head())
print("\nValue counts for Household_head_age_category:")
print(df_sel['Household_head_age_category'].value_counts(dropna=False))

In [ ]:
import numpy as np

df_sel['children_under5_category'] = np.where(
    df_sel['Number_children_under5'] <= 2,
    '<=2',
    '>2'
)

print(df_sel[['Number_children_under5', 'children_under5_category']].head())
print("\nValue counts for children_under5_category:")
print(df_sel['children_under5_category'].value_counts(dropna=False))

In [ ]:
df_sel.columns

In [ ]:
selected_final_columns = [
    'country_code_phase',
    'interview_year',
    'Residence',
    'Household_head_age_category',
    'children_under5_category',
    'household_members_category',
    'Has_electricity',
    'Has_radio',
    'Has_television',
    'Has_refrigerator',
    'household_head_sex',
    'Household_wealth_index',
    'Household_education_level',
    'Household_head_marital_status',
    'toilet_types',
    'distance_category',
    'sleeping_rooms_category',
    'source_drinking_water'
]

df_final = df_sel[selected_final_columns]
display(df_final.head())

In [ ]:
missing_values_percent_final = df_final.isnull().sum() / len(df_final) * 100
print("Percentage of missing values per column in df_final:")
print(missing_values_percent_final[missing_values_percent_final > 0].sort_values(ascending=False))

In [ ]:
for col in df_final.columns:
    if df_final[col].isnull().any():
        mode_value = df_final[col].mode()[0]  # Get the first mode if there are multiple
        df_final[col].fillna(mode_value, inplace=True)

print("Missing values after mode imputation:")
print(df_final.isnull().sum())

In [ ]:
df_final.head()

In [ ]:
for column in df_final.columns:
    print(f"\nProportions for '{column}':")
    print(df_final[column].value_counts(normalize=True, dropna=False))


In [ ]:
country_year_summary = df_final.groupby(['country_code_phase', 'interview_year']).size().reset_index(name='Sample Size')

total_sample_size = len(df_final)
country_year_summary['Percentage'] = (country_year_summary['Sample Size'] / total_sample_size) * 100

country_year_summary.rename(columns={'country_code_phase': 'Country', 'interview_year': 'Survey Year'}, inplace=True)

# Group by Country and aggregate years, sample size, and percentage
def format_year_range(years):
    if len(years) == 1:
        return str(years.iloc[0])
    min_year = years.min()
    max_year = years.max()
    if max_year == min_year + 1:
        return f"{min_year}/{str(max_year)[2:]}"
    else:
        return f"{min_year}-{max_year}"


country_summary_aggregated = country_year_summary.groupby('Country').agg(
    Survey_Year=('Survey Year', format_year_range),
    Sample_Size=('Sample Size', 'sum'),
    Percentage=('Percentage', 'sum')
).reset_index()

# Round the Percentage column to 2 decimal places
country_summary_aggregated['Percentage'] = country_summary_aggregated['Percentage'].round(2)

display(country_summary_aggregated.head())
print(country_summary_aggregated.to_string())

In [ ]:
csv_output_path = '/content/country_summary_aggregated.csv'
country_summary_aggregated.to_csv(csv_output_path, index=False)
print(f"Aggregated country summary saved to {csv_output_path}")

In [ ]:
df_final.columns

In [ ]:
df_final = df_final.rename(columns={
    'country_code_phase': 'country_code_phase',
    'interview_year': 'interview_year',
    'Residence': 'residence',
    'Household_head_age_category': 'household_head_age',
    'children_under5_category': 'Number_children_U5R',
    'household_members_category': 'Number_household_members',
    'Has_electricity': 'electricity_access',
    'Has_radio': 'Has_radio',
    'Has_television': 'Has_television',
    'Has_refrigerator': 'Has_refrigerator',
    'household_head_sex': 'Sex_of_household_head',
    'Household_wealth_index': 'household_wealth_index',
    'Household_education_level': 'household_head_education_level',
    'Household_head_marital_status': 'marital_status',
    'toilet_types': 'toilet_type',
    'distance_category': 'water_access_distance_in_Minutes',
    'sleeping_rooms_category': 'Number_sleeping_rooms',
    'source_drinking_water': 'drinking_water_source'
})


In [ ]:
df_final.columns

In [ ]:
education_mapping = {
    'no education, preschool': 'No formal Education',
    'don\'t know': 'No formal Education',
    '.a': 'No formal Education',
    'primary': 'Primary',
    'secondary': 'Secondary and Higher',
    'higher': 'Secondary and Higher'
}

df_final['household_head_education_level'] = df_final['household_head_education_level'].map(education_mapping)

print("Value counts for household_head_education_level after recategorization:")
print(df_final['household_head_education_level'].value_counts(dropna=False))

In [ ]:
marital_status_mapping = {
    'never married': 'Never married',
    'married': 'Married',
    'living together': 'Married',
    'not living together': 'Married',
    'divorced': 'Divorced/Widowed',
    'widowed': 'Divorced/Widowed'
}

df_final['marital_status'] = df_final['marital_status'].map(marital_status_mapping)

print("Value counts for marital_status after recategorization:")
print(df_final['marital_status'].value_counts(dropna=False))

In [ ]:
df_final.columns

In [ ]:
import pandas as pd

# -------------------------------------------
# List of variables to summarise
# -------------------------------------------
variables = [
    'residence', 'household_head_age', 'Number_children_U5R',
    'Number_household_members', 'electricity_access', 'Has_radio',
    'Has_television', 'Has_refrigerator', 'Sex_of_household_head',
    'household_wealth_index', 'household_head_education_level',
    'marital_status', 'toilet_type', 'water_access_distance_in_Minutes',
    'Number_sleeping_rooms'
]

# -------------------------------------------
# Function to generate one summary table
# -------------------------------------------
summary_tables = []

for var in variables:
    temp = (
        df_final
        .groupby([var, 'drinking_water_source'])
        .size()
        .unstack(fill_value=0)                 # columns = Improved / Unimproved
        .reset_index()
    )

    # Rename columns
    temp.rename(columns={
        var: 'Category',
        'Improved': 'Safe Water Access (Improved)',
        'Unimproved': 'Safe Water Access (Unimproved)'
    }, inplace=True)

    # Add Total column
    temp['Total'] = temp['Safe Water Access (Improved)'] + temp['Safe Water Access (Unimproved)']

    # Add variable name column
    temp.insert(0, 'Variable', var)

    summary_tables.append(temp)

# -------------------------------------------
# Combine into a single table
# -------------------------------------------
final_table = pd.concat(summary_tables, ignore_index=True)

# Display
final_table


In [ ]:
final_table.to_csv('safe_water_access_summary.csv', index=False)

print("CSV file saved as: safe_water_access_summary.csv")

In [ ]:
df_final.head()

In [ ]:
df_final.columns




In [ ]:
import pandas as pd

# Define custom label encoders (ordered categories)
label_maps = {
    'residence': {
        'rural': 0,
        'urban': 1
    },
    'household_head_age': {
        '<20': 0,
        '20-35': 1,
        '36-45': 2,
        '>46': 3
    },
    'Number_children_U5R': {
        '<=2': 0,
        '>2': 1
    },
    'Number_household_members': {
        '1-6': 0,
        '>6': 1
    },
    'electricity_access': {
        'no': 0,
        'yes': 1
    },
    'Has_radio': {
        'no': 0,
        'yes': 1
    },
    'Has_television': {
        'no': 0,
        'yes': 1
    },
    'Has_refrigerator': {
        'no': 0,
        'yes': 1
    },
    'Sex_of_household_head': {
        'female': 0,
        'male': 1
    },
    'household_wealth_index': {
        'poorest': 0,
        'poorer': 1,
        'middle': 2,
        'richer': 3,
        'richest': 4
    },
    'household_head_education_level': {
        'No formal Education': 0,
        'Primary': 1,
        'Secondary and Higher': 2
    },
    'marital_status': {
        'Never married': 0,
        'Divorced/Widowed': 1,
        'Married': 2
    },
    'toilet_type': {
        'Improved': 1,
        'Unimproved': 0
    },
    'water_access_distance_in_Minutes': {
        '<=30': 0,
        '>30': 1
    },
    'Number_sleeping_rooms': {
        '1': 0,
        '2': 1,
        '>=3': 2
    },
    'drinking_water_source': {
        # fill categories if needed
        'Improved': 1,
        'Unimproved': 0
    }
}

# Apply label encoding
for col, mapping in label_maps.items():
    df_final[col] = df_final[col].map(mapping)

# Display encoded values
for col, mapping in label_maps.items():
    print(f"\nLabel encoding for {col}:")
    print(mapping)
df_final.head()

In [ ]:
selected_model_columns = [
    'residence',
    'household_head_age',
    'Number_children_U5R',
    'Number_household_members',
    'electricity_access',
    'Has_radio',
    'Has_television',
    'Has_refrigerator',
    'Sex_of_household_head',
    'household_wealth_index',
    'household_head_education_level',
    'marital_status',
    'toilet_type',
    'water_access_distance_in_Minutes',
    'Number_sleeping_rooms',
    'drinking_water_source'
]

df_selected_for_model = df_final[selected_model_columns]
display(df_selected_for_model.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_selected_for_model already exists
try:
    df = df_selected_for_model.copy()
except NameError:
    raise NameError("df_selected_for_model is not found. Please upload or define it.")

# Compute correlation matrix
corr_matrix = df.corr(numeric_only=True)

# Set style for publication
sns.set(style="white")  # clean white background

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',  # professional diverging palette
    cbar=True,
    square=True,
    linewidths=0.5,   # subtle gridlines
    annot_kws={"size":10}  # adjust font size for annotation
)
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.yticks(rotation=0, fontsize=11)
plt.title("Correlation Heatmap", fontsize=16, weight='bold')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Assuming df_selected_for_model already exists
try:
    df = df_selected_for_model.copy()
except NameError:
    raise NameError("df_selected_for_model is not found. Please upload or define it.")

# Compute correlation matrix
corr_matrix = df.corr(numeric_only=True)

# Create graph
threshold = 0.5  # show only correlations above this absolute value
G = nx.Graph()

# Add edges for correlations above threshold
for i in corr_matrix.columns:
    for j in corr_matrix.columns:
        if i != j and abs(corr_matrix.loc[i, j]) >= threshold:
            G.add_edge(i, j, weight=corr_matrix.loc[i, j])

# Draw network
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, seed=42)  # positions for all nodes

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=2500, node_color='skyblue')

# Draw edges with width proportional to correlation
edges = G.edges()
weights = [abs(G[u][v]['weight'])*5 for u, v in edges]
nx.draw_networkx_edges(G, pos, width=weights, edge_color='gray')

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')

# Add title
plt.title("Correlation Network Graph (|corr| ≥ 0.5)", fontsize=16, weight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()
